# Defective Matrix and Schur Decomposition Analysis

This notebook demonstrates Schur decomposition

**Repository:** [https://github.com/bijesh-p/linear-algebra](https://github.com/bijesh-p/linear-algebra/blob/main/defective_matrix_schur.ipynb)

**Google Colab:** [Run in Colab](https://colab.research.google.com/github/bijesh-p/linear-algebra/blob/main/defective_matrix_schur.ipynb)



In [ ]:
import numpy as np
from scipy import linalg

# Set print options for better readability
np.set_printoptions(precision=6, suppress=True)

## Input Matrix

In [ ]:
# Define the matrix A
A = np.array([[-1, -1, -2],
              [8, -11, -8],
              [-10, 11, 7]])

print("Matrix A:")
print(A)

---

## Part (a): Checking if the Matrix is Defective

A matrix is **defective** if the algebraic multiplicity of at least one eigenvalue exceeds its geometric multiplicity.

- **Algebraic Multiplicity:** Number of times an eigenvalue appears as a root of the characteristic polynomial
- **Geometric Multiplicity:** Dimension of the eigenspace, i.e., $\dim(\text{null}(A - \lambda I))$

### Step 1: Compute Eigenvalues and Eigenvectors

In [ ]:
# Compute eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eig(A)

print("Eigenvalues:")
for i, lam in enumerate(eigenvalues):
    print(f"λ_{i+1} = {lam:.6f}")

### Step 2: Count Algebraic Multiplicities

In [ ]:
# Count algebraic multiplicities (with tolerance for numerical errors)
unique_eigenvalues, counts = np.unique(np.round(eigenvalues, decimals=3), return_counts=True)

print("\nAlgebraic Multiplicities:")
for val, count in zip(unique_eigenvalues, counts):
    print(f"λ = {val:.6f}, algebraic multiplicity = {count}")

### Step 3: Compute Geometric Multiplicities

For each eigenvalue $\lambda$, the geometric multiplicity is:
$$\text{geom. mult}(\lambda) = \dim(\text{null}(A - \lambda I)) = n - \text{rank}(A - \lambda I)$$

In [ ]:
# Check geometric multiplicities by finding null space dimensions
print("\nGeometric Multiplicities:")
geometric_multiplicities = []

for i, lam in enumerate(unique_eigenvalues):
    # For eigenvalue λ, geometric multiplicity = dim(null(A - λI))
    A_minus_lambdaI = A - lam * np.eye(3)
    
    # Compute rank
    rank = np.linalg.matrix_rank(A_minus_lambdaI, tol=1e-10)
    
    # Geometric multiplicity = n - rank(A - λI)
    geom_mult = 3 - rank
    geometric_multiplicities.append(geom_mult)
    
    print(f"λ = {lam:.6f}, geometric multiplicity = {geom_mult}")

### Step 4: Check if Matrix is Defective

In [ ]:
# Check if defective
is_defective = False
print("\nChecking if matrix is defective:")
print("-" * 70)

for i, (val, alg_mult, geom_mult) in enumerate(zip(unique_eigenvalues, counts, geometric_multiplicities)):
    if alg_mult > geom_mult:
        print(f"✓ λ = {val:.6f}: algebraic mult ({alg_mult}) > geometric mult ({geom_mult})")
        print(f"  This eigenvalue is DEFECTIVE!")
        is_defective = True
    else:
        print(f"  λ = {val:.6f}: algebraic mult ({alg_mult}) = geometric mult ({geom_mult})")

print("\n" + "=" * 70)
if is_defective:
    print("*** CONCLUSION: Matrix A is DEFECTIVE ***")
else:
    print("*** Matrix A is NOT defective ***")
print("=" * 70)

### Step 5: Check Diagonalizability

A matrix is diagonalizable if and only if the sum of geometric multiplicities equals $n$ (the matrix dimension).

In [ ]:
# Check diagonalizability
print("\nChecking diagonalizability:")
total_geom_mult = sum(geometric_multiplicities)
print(f"Sum of geometric multiplicities: {total_geom_mult}")
print(f"Matrix size: 3")

if total_geom_mult < 3:
    print("\n*** Matrix A is NOT DIAGONALIZABLE ***")
    print("(Because sum of geometric multiplicities < n)")
else:
    print("\n*** Matrix A is DIAGONALIZABLE ***")

---

## Part (b): Computing Schur Decomposition

The **Schur decomposition** of a matrix $A$ is:
$$A = UTU^H$$

where:
- $U$ is a unitary matrix ($U^H U = I$)
- $T$ is an upper triangular matrix
- The diagonal elements of $T$ are the eigenvalues of $A$

We'll compute the Schur decomposition using two methods: (1) Manual implementation following the class procedure, and (2) Built-in scipy function. 
Since the schur demposition may not be unique , the verification is done by reconstructing original matrix and checking the properties 

In [ ]:
def verify_schur(U, T, A, method_name=""):
    """
    Verify that U and T form a valid Schur decomposition of A.
    
    Checks:
    1. U is unitary: U^H U = I
    2. T is upper triangular
    3. A = U T U^H (reconstruction)
    4. Diagonal of T contains eigenvalues of A
    
    Parameters:
    -----------
    U : ndarray
        Unitary matrix
    T : ndarray
        Upper triangular matrix
    A : ndarray
        Original matrix
    method_name : str
        Name of the method for display
    """
    print("=" * 70)
    print(f"SCHUR DECOMPOSITION VERIFICATION - {method_name}")
    print("=" * 70)
    
    # Check 1: U is unitary
    print("\n1. Checking if U is unitary (U^H U = I):")
    U_conj_T = np.conj(U.T)
    UH_U = U_conj_T @ U
    identity_error = np.max(np.abs(UH_U - np.eye(len(U))))
    print(f"   Max |U^H U - I|: {identity_error:.2e}")
    if identity_error < 1e-10:
        print("   ✓ U is unitary")
    else:
        print("   ✗ U is NOT unitary")
    
    # Check 2: T is upper triangular
    print("\n2. Checking if T is upper triangular:")
    below_diag = np.max(np.abs(np.tril(T, -1)))
    print(f"   Max |element below diagonal|: {below_diag:.2e}")
    if below_diag < 1e-10:
        print("   ✓ T is upper triangular")
    else:
        print("   ✗ T is NOT upper triangular")
    
    # Check 3: Reconstruction A = U T U^H
    print("\n3. Checking reconstruction (A = U T U^H):")
    reconstructed = U @ T @ U_conj_T
    reconstruction_error = np.max(np.abs(A - reconstructed.real))
    print(f"   Max |A - U T U^H|: {reconstruction_error:.2e}")
    if reconstruction_error < 1e-10:
        print("   ✓ Reconstruction is accurate")
    else:
        print("   ✗ Reconstruction has errors")
    
    # Check 4: Eigenvalues on diagonal
    print("\n4. Checking eigenvalues on diagonal of T:")
    diag_T = np.diag(T)
    eigenvalues_A = np.linalg.eigvals(A)
    # Compare sets of eigenvalues (sort by real part then imag part)
    diag_sorted = np.sort_complex(diag_T)
    eig_sorted = np.sort_complex(eigenvalues_A.astype(complex))
    eigenvalue_error = np.max(np.abs(diag_sorted - eig_sorted))
    print(f"   Diagonal of T: {diag_T}")
    print(f"   Eigenvalues of A: {eigenvalues_A}")
    print(f"   Max difference: {eigenvalue_error:.2e}")
    if eigenvalue_error < 1e-6:
        print("   ✓ Eigenvalues match")
    else:
        print("   ✗ Eigenvalues do NOT match")
    
    # Overall result
    print("\n" + "=" * 70)
    all_passed = (identity_error < 1e-10 and below_diag < 1e-10 and 
                  reconstruction_error < 1e-10 and eigenvalue_error < 1e-6)
    if all_passed:
        print("✓✓✓ ALL CHECKS PASSED - Valid Schur Decomposition ✓✓✓")
    else:
        print("✗✗✗ SOME CHECKS FAILED - Invalid Schur Decomposition ✗✗✗")
    print("=" * 70 + "\n")
    
    return all_passed
    print("=" * 70 + "\n")
    
    return all_passed

### Method 1: Manual Schur Decomposition (Class Procedure)

**Algorithm (from class notes):**
1. Find an eigenpair $(\lambda_1, x_1)$ where $Ax_1 = \lambda_1 x_1$
2. Normalize $x_1$ to get a unit vector
3. Extend $\{x_1\}$ to an orthonormal basis using Gram-Schmidt
4. Form the unitary matrix $U_1 = [x_1 | v_2 | v_3]$
5. Compute $U_1^H A U_1$ - this gives block upper triangular form with $\lambda_1$ in top-left
6. Recursively apply the procedure to the bottom-right $(n-1) \times (n-1)$ block
7. Combine the transformations to get full Schur form

In [ ]:
print("Step 1: Find first eigenpair (λ₁, x₁)")
print("-" * 70)

# Find the first eigenvalue and eigenvector
idx = 0  # We'll use the first eigenvalue
lambda_1 = eigenvalues[idx]
x_1 = eigenvectors[:, idx]

print(f"λ₁ = {lambda_1:.6f}")
print(f"x₁ = {x_1}")
print()

# Normalize x_1
x_1 = x_1 / np.linalg.norm(x_1)
print(f"Normalized x₁ = {x_1}")

In [ ]:
print("\nStep 2: Extend x₁ to orthonormal basis using Gram-Schmidt")
print("-" * 70)

def gram_schmidt(vectors):
    """Apply Gram-Schmidt orthogonalization"""
    basis = []
    for v in vectors:
        # Orthogonalize against existing basis
        w = v.copy()
        for b in basis:
            w = w - np.dot(np.conj(b), w) * b
        
        # Check if vector is non-zero (linearly independent)
        if np.linalg.norm(w) > 1e-10:
            # Normalize
            w = w / np.linalg.norm(w)
            basis.append(w)
    
    return np.array(basis).T

# Create candidate vectors: start with x_1, then add standard basis vectors
candidate_vectors = [x_1]
for i in range(3):
    e_i = np.zeros(3, dtype=complex)
    e_i[i] = 1.0
    candidate_vectors.append(e_i)

# Apply Gram-Schmidt
U_manual = gram_schmidt(candidate_vectors)

print("Unitary matrix U (manual construction):")
print(U_manual)

In [ ]:
# Verify U is unitary
print("\nVerification that U is unitary:")
print("U^H U =")
print(np.conj(U_manual.T) @ U_manual)

In [ ]:
print("\nStep 3: Compute U_1^H A U_1 (first transformation)")
print("-" * 70)

T_step1 = np.conj(U_manual.T) @ A @ U_manual

print("After first transformation U_1^H A U_1:")
print(T_step1)
print()
print(f"Note: This has λ₁ = {T_step1[0,0]:.6f} in top-left")
print(f"The (2×2) block in bottom-right needs further triangularization:")
print(T_step1[1:, 1:])

In [ ]:
print("\nStep 4: Apply procedure to the (2×2) bottom-right block")
print("-" * 70)

# Extract the 2×2 block
A_2 = T_step1[1:, 1:]
print("2×2 block A_2:")
print(A_2)

# Find eigenpair for this block
eigenvalues_2, eigenvectors_2 = np.linalg.eig(A_2)
print(f"\nEigenvalues of A_2: {eigenvalues_2}")

# Take first eigenvector and normalize
x_2 = eigenvectors_2[:, 0]
x_2 = x_2 / np.linalg.norm(x_2)
print(f"Normalized eigenvector: {x_2}")

# Extend to orthonormal basis for 2×2
candidate_2 = [x_2, np.array([1.0, 0.0], dtype=complex)]
U_2_small = gram_schmidt(candidate_2)
print(f"\nUnitary matrix for 2×2 block:")
print(U_2_small)

# Embed into 3×3 identity
U_2 = np.eye(3, dtype=complex)
U_2[1:, 1:] = U_2_small

print(f"\nFull 3×3 matrix U_2 (identity in top-left):")
print(U_2)

In [ ]:
print("\nStep 5: Complete Schur form by combining transformations")
print("-" * 70)

# Combine: U_manual (first step) and U_2 (second step)
# Final U = U_manual @ U_2
U_manual_complete = U_manual @ U_2

print("Final unitary matrix U = U_1 @ U_2:")
print(U_manual_complete)

# Compute final T
T_manual = np.conj(U_manual_complete.T) @ A @ U_manual_complete

print("\nFinal upper triangular matrix T = U^H A U:")
print(T_manual)
print()


In [ ]:
# Verify the manual Schur decomposition
verify_schur(U_manual_complete, T_manual, A, method_name="Manual Method (Class Procedure)")

## Method 2: Built-In Method (linalg.schur)

In [ ]:
# Using built-in function
T_builtin, U_builtin = linalg.schur(A, output='complex')

print("Unitary matrix U (from built-in):")
print(U_builtin)
print()

print("Upper triangular matrix T = U^H A U (from built-in):")
print(T_builtin)

# Verify the manual Schur decomposition
verify_schur(U_builtin, T_builtin, A, method_name="Built-in Method")